In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import MaskedAutoencoderViT
from torch.utils.data import DataLoader
import torch 
import lightning.pytorch as pl
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!export CUDA_VISIBLE_DEVICES=0
import wandb
from data import DummyDataset

In [3]:
dataset = DummyDataset()

In [6]:
model = MaskedAutoencoderViT(img_size=(65, 4000), \
                                       patch_size=(5, 10), \
                                       in_chans=1)

dataset = DummyDataset()
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# dataset = DummyDataset()
# # batch_size = 1
# dataloader = DataLoader(dataset)

# for batch in dataset: 
#     inputs, targets = batch 
#     print(inputs.shape)

torch.Size([1, 65, 4000])


In [31]:
# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/ --port 7000

In [7]:
wandb.login()
logger = pl.loggers.WandbLogger(project="brain-age", name="mae_overfitting_EEG_sample_AdamW_optim", 
                                save_dir="/u/home/elbs/brain-age/drafts/elbaroudy", log_model=False)

# early_stop_callback = EarlyStopping(monitor="validation loss", min_delta=0.00, patience=3, verbose=False, mode="max")

trainer = pl.Trainer(overfit_batches=1.0, max_epochs=100, accelerator="gpu", logger=logger, devices=1)
trainer.fit(model=model, train_dataloaders=dataloader)
# trainer.fit(model, dataloader_train, dataloader_val)
wandb.finish()

wandb: Currently logged in as: sama-elbaroudy (brain-age). Use `wandb login --relogin` to force relogin


/u/home/elbs/.conda/envs/brain-age/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /u/home/elbs/.conda/envs/brain-age/lib/python3.11/si ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1.0)` was configured so 100% of the batches will be used..
/u/home/elbs/.conda/envs/brain-age/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /u/home/elbs/.conda/envs/brain-age/lib/python3.11/si ...


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss_epoch,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_step,█▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch,99
loss_epoch,0.02745
loss_step,0.02745
trainer/global_step,99


In [9]:
import matplotlib.pyplot as plt 
print(model)

MaskedAutoencoderViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 1024, kernel_size=(5, 10), stride=(5, 10))
  )
  (blocks): ModuleList(
    (0-23): 24 x Block(
      (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (decoder_embed): Linear(in_features=1024, out_features=51

In [ ]:


# trainer = pl.Trainer(limit_train_batches=100, max_epochs=100, devices=1)

# trainer.fit(model=MaskedAutoencoderViT(img_size=(65, 650), \
#                                        patch_size=(5, 10), \
#                                        in_chans=1), \
#             train_dataloaders=dataloader)

In [6]:
from typing import Iterable
import misc as misc
import lr_sched as lr_sched
def train_one_epoch(model: torch.nn.Module,
                    data_loader: Iterable, optimizer: torch.optim.Optimizer,
                    device: torch.device, epoch: int, loss_scaler,
                    log_writer=None,
                    args=None):
    model.train(True)
    metric_logger = misc.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 20

    accum_iter = args.accum_iter

    optimizer.zero_grad()

    if log_writer is not None:
        print('log_dir: {}'.format(log_writer.log_dir))

    for data_iter_step, (samples, _) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):

        # we use a per iteration (instead of per epoch) lr scheduler
        if data_iter_step % accum_iter == 0:
            lr_sched.adjust_learning_rate(optimizer, data_iter_step / len(data_loader) + epoch, args)

        samples = samples.to(device, non_blocking=True)

        with torch.cuda.amp.autocast():
            loss, _, _ = model(samples, mask_ratio=args.mask_ratio)

        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            sys.exit(1)

        loss /= accum_iter
        loss_scaler(loss, optimizer, parameters=model.parameters(),
                    update_grad=(data_iter_step + 1) % accum_iter == 0)
        if (data_iter_step + 1) % accum_iter == 0:
            optimizer.zero_grad()

        torch.cuda.synchronize()

        metric_logger.update(loss=loss_value)

        lr = optimizer.param_groups[0]["lr"]
        metric_logger.update(lr=lr)

        loss_value_reduce = misc.all_reduce_mean(loss_value)
        if log_writer is not None and (data_iter_step + 1) % accum_iter == 0:
            """ We use epoch_1000x as the x-axis in tensorboard.
            This calibrates different curves when batch size changes.
            """
            epoch_1000x = int((data_iter_step / len(data_loader) + epoch) * 1000)
            log_writer.add_scalar('train_loss', loss_value_reduce, epoch_1000x)
            log_writer.add_scalar('lr', lr, epoch_1000x)


    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

In [7]:
model = MaskedAutoencoderViT(img_size=(65, 650), \
                                       patch_size=(5, 10), \
                                       in_chans=1)

dataset = DummyDataset()
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
from misc import NativeScalerWithGradNormCount as NativeScaler
loss_scaler = NativeScaler
from torch.utils.tensorboard import SummaryWriter
log_writer = SummaryWriter(log_dir='log')
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.95))
class myargs(): 
    def __init__(self): 
        self.mask_ratio = 0.75
        self.accum_iter = 1 
        self.warmup_epochs = 40
        self.lr = 0.01
args = myargs()
# print(args.accum_iter)
train_one_epoch(model, dataloader, optimizer, device, 1, loss_scaler, log_writer, args)

log_dir: log
torch.Size([1, 1, 65, 650])


RuntimeError: Input type (c10::Half) and bias type (float) should be the same

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print("Current device:", device)

In [ ]:
# import matplotlib.pyplot as plt
# x = model.pos_embed
# print(x.squeeze(0).shape)
# attn = torch.matmul(x.squeeze(0), x.squeeze(0).T)
# # attn[torch.eye(attn.shape[0], dtype=bool)] = 0

# # plt.plot(attn[0][torch.argsort(attn[0])])
# plt.imshow(attn)
# plt.show()

In [ ]:
# x = model.pos_embed
# for block in model.blocks:
#     x = block(x)

In [ ]:
# model(dummy_EEG_input)

In [ ]:
# device_count = torch.cuda.device_count()
# print(device_count)